## Black-Scholes PDE
The `Black-Scholes partial differential equation (PDE)` for option $V$ is:
$$
\frac{\partial V}{\partial t} + r S_t \frac{\partial V}{\partial S} +  \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 V}{\partial S^2} - r V_t = 0 $$
Writing $\Delta_t=\frac{\partial V}{\partial S_t}$ and $\Gamma_t=\frac{\partial^2 V}{\partial S_t^2}$, we get
$$\frac{\partial V}{\partial t} + r S_t \Delta_t + \frac{1}{2} \sigma^2 S_t^2 \Gamma_t - r V_t = 0 
$$

## The Greeks
We can measure the sensitivity of the price to market conditions using partial derivatives of $V$.
These partial derivatives are collectively called `The Greeks.` These can be seen as some kind of Risk-measure since they measure sensitivity to changes in the market. As the partial derivatives are linear, we can add up the Greeks of the components to compute the Greeks of the portfolio.
If a trader writes (i.e. sells) a call option and replicates it using the delta-hedging strategy, then the delta of their portfolio is 0 at all times. This is because their portfolio consists of $-1$ units of the option and $\Delta$ units of the stock. So the total $\Delta$ is $-1 \cdot  \Delta + 1 \cdot \Delta=0$, since the $\Delta$ of the stock is 1 ($\frac{\partial S}{\partial S}=1$). So the $\Delta$ hedging strategy can be viewed
as the strategy of ensuring that this particular risk-measure is always 0. In practice, derivatives traders manage their portfolios to ensure that most of the Greek's are low as, at least heuristically, one would then expect that the portfolio is hedged against risks such as a change in interest rate or volatility.
$$
\begin{split}
\Delta&:=\frac{\partial V}{\partial S} \\ 
\Gamma&:=\frac{\partial^2 V}{\partial S^2} \\
\rho&:=\frac{\partial V}{\partial r} \\
\theta&:=-\frac{\partial V}{\partial t} \\
\text{Vega}&:=\frac{\partial V}{\partial \sigma}
\end{split}
$$

### Calculating formula for $\Delta$ using `sympy`
We begin by creating the symbols we need and then compute $V$ according to the Black-Scholes formula.
We introduce a variable $\tau=T-t$. We also indicate that all our variables will be positive as this will allow more simplification rules to be applied later. Since $N$ is a symbolic function rather than a variable, we must create it using `Function` command instead of using `symbols`.

In [4]:
import sympy as sp

S, tau, K, r, sigma, D1, D2 = sp.symbols('S tau K r sigma D1 D2',positive=True)
N = sp.Function('N')
d1 = 1/(sigma*sp.sqrt(tau))*(sp.log(S/K)+(r+sigma**2/2)*tau)
d2 = d1 - sigma * sp.sqrt(tau)
V = S*N(d1) -K*sp.exp(-r*tau)*N(d2)
display(V)
Delta = sp.diff(V,S)
Delta

-K*N(-sigma*sqrt(tau) + (tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))*exp(-r*tau) + S*N((tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))

-K*exp(-r*tau)*Subs(Derivative(N(_xi_1), _xi_1), _xi_1, -sigma*sqrt(tau) + (tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))/(S*sigma*sqrt(tau)) + N((tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau))) + Subs(Derivative(N(_xi_1), _xi_1), _xi_1, (tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))/(sigma*sqrt(tau))

Notice that we obtain an expression for $\Delta$ in terms of the function $N$ and its partial derivative. We know the partial derivative of $N$ satisfies
$$
\frac{d N(x)}{dx}=\frac{1}{\sqrt{2 \pi}}\exp{-x^2/2}
$$
so we can simplify these derivative terms. What we want to do is take our expression for $\Delta$ and replace all partial derivatives of the form
$$
\frac{d N(*_x)}{d *_x}
$$
with
$$
\frac{1}{\sqrt{2 \pi}}e^{-\frac{*_x^2}{2}}.
$$
In these last two expressions $*_x$ could be any variable at all, but it must be the same expression each time it occurs. The label $x$ on
the pattern indicates that all the $*_x$ expressions must be identical. You can make changes to a `sympy` expression using the `replace` function. 

In [10]:
Delta = Delta.replace(sp.Derivative(N(sp.Wild('x')),sp.Wild('x')), sp.exp(-sp.Wild('x')**2/2)/(sp.sqrt(2*sp.pi)))
Delta = Delta.simplify()
Delta 

N((2*r*tau + sigma**2*tau + log(S**2/K**2))/(2*sigma*sqrt(tau)))

This can be written very neatly as
$$\Delta = N(d_1).$$
We check this on the next line to make sure by simlifying $\Delta - N(d_1)$. We expect zero.

In [11]:
(N(d1)-Delta).simplify()

0

- Lets use `sympy` to show that $V$ satisfies the Black-Scholes PDE
$$
\frac{\partial V}{\partial t} + r S \frac{\partial V}{\partial S } + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2 } - r V = 0.
$$

In [12]:
Gamma = sp.diff( Delta, S )
Gamma = Gamma.replace(sp.Derivative(N(sp.Wild('x')),sp.Wild('x')), sp.exp(-sp.Wild('x')**2/2)/(sp.sqrt(2*sp.pi)))
Gamma = Gamma.simplify()
Gamma

sqrt(2)*exp(-(2*r*tau + sigma**2*tau + log(S**2/K**2))**2/(8*sigma**2*tau))/(2*sqrt(pi)*S*sigma*sqrt(tau))

In [14]:
theta = -sp.diff(V, tau)
theta = theta.replace(sp.Derivative(N(sp.Wild('x')),sp.Wild('x')), sp.exp(-sp.Wild('x')**2/2)/(sp.sqrt(2*sp.pi)))
theta = theta.simplify()
theta

-(4*K*r*sigma*tau**(9/2)*N((2*r*tau - sigma**2*tau + log(S**2/K**2))/(2*sigma*sqrt(tau))) + sqrt(2)*K*tau**3*(2*sigma**2*tau - tau*(2*r + sigma**2) + log(S**2/K**2))*exp(-(-2*sigma**2*tau + tau*(2*r + sigma**2) + log(S**2/K**2))**2/(8*sigma**2*tau))/(2*sqrt(pi)) + sqrt(2)*S*tau**3*(tau*(2*r + sigma**2) - log(S**2/K**2))*exp(r*tau)*exp(-(tau*(2*r + sigma**2) + log(S**2/K**2))**2/(8*sigma**2*tau))/(2*sqrt(pi)))*exp(-r*tau)/(4*sigma*tau**(9/2))

In [16]:
black_scholes_pde = theta + r*S*Delta + sigma**2*S**2*Gamma/2 - r*V
black_scholes_pde = black_scholes_pde.simplify()
assert black_scholes_pde == 0
black_scholes_pde

0

## Delta
Measures the exposure to the `directional risk`. Delta is the rate of change of the option value with respect to change in the underlying asset price. Its typically expressed in terms of the amount in options position value will change for 1% move in the price of the underlying (estimation). Delta is the linear approximation of curved option payoff function.
$$
\Delta = \frac{\partial V}{\partial S}\\
\Delta_{\text{call}} = \Phi(d_1)\\
\Delta_{\text{put}} = -\Phi(-d_1)
$$
If $\Delta$ of call option is $0.7$, you expect the call price to rise by $0.70$ pence for every £1 rise in the price of underlying stock.
- $\Delta$ is the slope of tangent line to option valuation curve.
- $\Delta$ of a vanilla call option can only be in the range $[0,1]$.
- $\Delta$ falls at low spot prices$(S)$ relative to strike$(K)$ of the option and increases at high spot prices relative to strike.
- $\Delta$ is always *positive* for *long call* and *negative* for *long put* option. Its *negative* for *short call* option and *positive* for *short put* option.
- Call option value *increases* as *stock price increases* and *put option decreases.*
- For European Call and Put with same $K,T$ and $S$ with *no dividend yield* $$\Delta_{\text{call}} - \Delta_{\text{put}} = 1$$ This is due to *Put-Call Parity*. i.e. Long call + short put replicates a Forward contract which is $\Delta = 1$.
- $\Delta$ tells us how long or short your portfolio is at a given point in time.
- $\Delta$ of a portfolio of options on same underlying asset equals sum of $\Delta$ for each individual option positions. Since, the $\Delta$ of underlying asset is always 1, a Trader could $\Delta$ hedge the entire position in the underlying by buying or shorting the number of shares indicated by the Total portfolio's $\Delta$.

`Delta Neutral Portfolio`: This is the portfolio made up of positions with offsetting positive and negative $\Delta$s such that the net $\Delta$ of the portfolio is zero. Such portfolio contains options and their corresponding underlying securities such that the positive and negative $\Delta$ components offset each other. The key idea is to make the portfolio insensitive to changes in value of underlying security.

In [23]:
import numpy as np
import mathfinance as mf
from scipy.stats import norm

S0 = 100      
K = 100      
T = 1     # 1 Year.
t = 0
r = 0.06           
sigma = 0.40

blackscholes_call_price = mf.blackscholes(r, S0, K, T, sigma, t=0, type="call")
delta_call = mf.black_scholes_call_delta(S0,t,K,T,r,sigma)
delta_put = mf.black_scholes_put_delta(S0,t,K,T,r,sigma)
print(blackscholes_call_price, delta_call, delta_put)

18.472604456409655 0.6368306511756191 -0.3631693488243809


## Gamma
Measures the exposure to `price fluctuations` in either direction. Gamma is the rate of change in the delta with respect to changes in the underlying price. 
$$
\Gamma = \frac{\partial \Delta}{\partial S} = \frac{\partial^2 V}{\partial S^2}\\
\Gamma = \frac{\phi(d_1)}{S \sigma \sqrt{\tau}}
$$
This is a second order greek. It measures how much $\Delta$ changes as a 1% change in the price of an underlying.
- Small $\Gamma$ implies $\Delta$ changes slowly and delta-neutral hedging is not required quite as often. For larger $\Gamma$, $\Delta$ is very sensitive to changes in price of underlying and frequent hedging is required for delta-neutral position.
- All long options have positive $\Gamma$ and all short options have negative $\Gamma$.
- $\Gamma$ is largest at *Near-The-Money* options. It diminishes as further in or out of the money you go.
- $\Gamma$ varies with $T$. So for *At-The-Money (ATM)* option, $\Gamma$ increases as expiration approaches. For options that are deep in or out of the money, $\Gamma$ will fall substantially as expiration approaches.
- $\Gamma$ helps to understand how our portfolio will behave for larger movements in underlying. $\Delta$ is not as informative for larger movements.
- $\Delta$-Neutrality provides protection against small movements in price of underlying while $\Gamma$-Neutrality provides protection against larger movements between rebalancing. The closer your option is to being *ATM*, the higher the option $\Gamma$ will be.
- $\Gamma$ is affected by passage of time differently depending on the options moneyness. If option is *ATM*, $\Gamma$ increases significantly as you get closer to expiry. As you get further out the money or deep in the money the effect on $\Gamma$ is minimised.
- The further out in time your expiration, the smaller the $\Gamma$.
- $\Gamma$ and $\Delta$ will not move around significantly if you have 3,6 or 9 months left to expiry even if the option moves in the money.

In [24]:
def black_scholes_call_gamma(S, t, K, T, r):
    "Calculate gamma of a option"
    tau = T-t
    d1 = (np.log(S/K) + (r + sigma**2/2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    gamma = norm.pdf(d1, 0, 1)/(S*sigma*np.sqrt(tau))
    return gamma

gamma = black_scholes_call_gamma(S0, t, K, T, r)
print(gamma)

0.009381008672923448


## Vega
Vega measures sensitivity to volatility (i.e. measures exposure to changes in `Implied Volatility`). Vega is the rate of change of the option value with respect to change in the underlying asset volatility.
$$
\text{Vega} = \frac{\partial V}{\partial \sigma} \\
\text{Vega} = S \phi(d_1) \sqrt{\tau}
$$

In [25]:
def black_scholes_call_vega(S, t, K, T, r):
    "Calculate vega of a option"
    tau = T-t
    d1 = (np.log(S/K) + (r + sigma**2/2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    vega = 0.01 *S*norm.pdf(d1, 0, 1)*np.sqrt(tau)  # 1% change in sigma.
    return vega

vega = black_scholes_call_vega(S0, t, K, T, r)
print(vega)

0.3752403469169379


## Theta
Measures the exposure to the `passage of time`. Theta measures the sensitivity of the value of the Option to the passage of time - time decay.
$$
\Theta = -\frac{\partial V}{\partial \tau} \\
\Theta_{call} = -\frac{S\phi(d_1)\sigma}{2\tau} - rK\exp{(-r\tau)}\Phi(d2)\\
\Theta_{put} = -\frac{S\phi(d_1)\sigma}{2\tau} + rK\exp{(-r\tau)}\Phi(-d2)
$$
Option price falls due to passage of time. Traders usually look at one day $\Theta$, it tells how much theoretically option value will fall in price with all other things held equal if one day passes. We cannot hedge $\Theta$.

In [26]:
def black_scholes_call_theta(S, t, K, T, r):
    "Calculate theta of a call option"
    tau = T-t
    d1 = (np.log(S/K) + (r + sigma**2/2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    theta = -S*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(tau)) - r*K*np.exp(-r*tau)*norm.cdf(d2, 0, 1)
    theta = theta/365    # time decay per day.
    return theta

def black_scholes_put_theta(S, t, K, T, r):
    "Calculate theta of a put option"
    tau = T-t
    d1 = (np.log(S/K) + (r + sigma**2/2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    theta = -S*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(tau)) + r*K*np.exp(-r*tau)*norm.cdf(-d2, 0, 1)
    theta = theta/365
    return theta

theta_call = black_scholes_call_theta(S0, t, K, T, r)
theta_put = black_scholes_put_theta(S0, t, K, T, r)
print(theta_call)
print(theta_put)

-0.027992971446596973
-0.012511910620554526


## Rho
Rho measures the sensitivity to the interest rate(i.e. Measures the exposure to changes in the `risk-free interest` rate.).
$$
\rho = \frac{\partial V}{\partial r} \\
\rho_{call} = K\tau\exp{(-r\tau)}\Phi(d2)\\
\rho_{put} = -K\tau\exp{(-r\tau)}\Phi(-d2)
$$
$\rho$ is usually expressed as how much option price change if the risk-free rate rises or falls by 1% per annum.
- Increase in risk-free rate(r) will increase value of the call option and decreases put option value.
- $\rho$ is *positive* for *long call* and *negative* for *long put.*
- $\rho$ changes as time passes moving towards zero as expiration approaches since present valuing effect in our options becomes less and less as we get closer to maturity.
- $\rho$ is low for call options thats deep *Out-The-Money(OTM)* and high for deep *In-The-Money(ITM)*. $\rho$ is high for deep *OTM* put and low for deep *ITM* put.
- $\rho$ is the least used of all the Greeks.

In [27]:
def black_scholes_call_rho(S, t, K, T, r):
    "Calculate rho of a call option"
    tau = T-t
    d1 = (np.log(S/K) + (r + sigma**2/2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    rho = K*tau*np.exp(-r*tau)*norm.cdf(d2, 0, 1)
    return 0.01*rho    # 1% change in Interest rate.

def black_scholes_put_rho(S, t, K, T, r):
    "Calculate rho of a put option"
    tau = T-t
    d1 = (np.log(S/K) + (r + sigma**2/2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    rho = -K*tau*np.exp(-r*tau)*norm.cdf(-d2, 0, 1)
    return 0.01*rho

rho_call = black_scholes_call_rho(S0, t, K, T, r)
rho_put = black_scholes_put_rho(S0, t, K, T, r)
print(rho_call)
print(rho_put)

0.4521046066115226
-0.48965992697272614


### Neutral Portfolio
- We can create $\Delta$- Neutralportfolio using stock and option. We can create $\Gamma$-Neutral portfolio by adding another option.
- A portfolio of a stock and a single option can never be $\Gamma$-Neutral.
- Greeks are additive in a linear manner i.e. Total $\Delta$ exposure for all options for a given underlying equals to sum of $\Delta$ of each individual options.
- Example: Find the Net $\Delta$ of the following portfolio.

  Short 200 shares, long 5 Calls with $\Delta = 0.5$ and short 2 Puts with $\Delta = -0.2$. Note: Each option covers 100 shares.
  $$ \text{Net } \Delta = -200 + (5 \times 100 \times 0.5) - (2 \times 100 \times -0.2) = 90$$ Note, each underlying has $\Delta =1$, so we used 200. This is equivalent to 90 shares. So owning this portfolio gives you risk exposure as if you were long 90 shares of the underlying. This approach only works for $\Delta , \Gamma$ and $\Theta$.